### This will work for Google Colab

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForPreTraining
import torch

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')
import io
with open('data.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
len(text)

In [ ]:
text[:3]

In [ ]:
text[-4:]

In [ ]:
bag = [item for sentence in text for item in sentence.split('.') if item != '']

In [ ]:
bag[0]

In [ ]:
len(bag)

In [3]:
len(max(bag, key = len))

NameError: name 'bag' is not defined

In [ ]:
#bag = bag[:50]

In [ ]:
test = bag[-4:]

In [ ]:
bag = bag[:-4]
bag_size = len(bag)

In [ ]:
test_bag_size = len(test)

In [ ]:
test_bag_size

In [ ]:
bag_size

In [ ]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)
            


In [ ]:
sentence_a

In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=100, truncation=True, padding='max_length')

In [ ]:
input.keys()

In [ ]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
            (inputs.input_ids != 102) * (inputs.input_ids != 0)


In [ ]:
selection = []
for i in range(inputs.input_ids.shape[0]):
    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
input.keys()

In [ ]:
class MeditationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW
# activating training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from tqdm.notebook import tqdm # for our progress bar

epochs = 5

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        #initialize calculated gradients (from prev steps)
        optim.zero_grad()
        # pull all tensor barches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        #process
        outputs = model(input_ids, attention_mask = attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        #extract loss
        loss = outputs.loss
        #calculate loss for every paramter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        #print relevant info to progress bar
        loop.set_description(f'Epoch{epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
test

In [ ]:
test[0]

In [ ]:
test[1]

In [ ]:
inputs = tokenizer(test[0], test[2], return_tensors='pt')
inputs.keys()

In [ ]:
outputs = model(**inputs)
outputs.keys()

In [ ]:
torch.argmax(outputs.seq_relationship_logits)